### Create awards from backfill table `openalex.mid.work_funder`

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.backfill_awards
USING delta
AS
WITH awards AS (
  SELECT DISTINCT funder_id, lower(award_id) as normalized_award_id, award_id as funder_award_id
  FROM openalex.mid.work_funder
  LATERAL VIEW EXPLODE(award_ids) AS award_id
),
funders AS (
  SELECT 
    funder_id,
    display_name,
    ror_id,
    doi
  FROM openalex.common.funder
)
SELECT 
  -- Generate unique ID using xxhash64 of funder_id:normalized_award_id
  abs(xxhash64(concat(f.funder_id, ':', a.normalized_award_id))) % 9000000000 as id,
  
  -- Display name (not available in this source)
  CAST(NULL AS STRING) as display_name,
  
  -- Description (not available in this source)
  CAST(NULL AS STRING) as description,
  
  -- Funder info
  f.funder_id,
  a.funder_award_id,
  
  -- Amount (not available in this source)
  CAST(NULL AS DOUBLE) as amount,
  CAST(NULL AS STRING) as currency,
  
  -- Funder struct
  struct(
    concat('https://openalex.org/F', f.funder_id) as id,
    f.display_name,
    f.ror_id,
    f.doi
  ) as funder,
  
  -- Funding type (not available in this source)
  CAST(NULL AS STRING) as funding_type,
  
  -- Funder scheme (not available in this source)
  CAST(NULL AS STRING) as funder_scheme,
  
  -- Provenance
  'crossref_work.grants' as provenance,
  
  -- Dates (not available in this source)
  CAST(NULL AS DATE) as start_date,
  CAST(NULL AS DATE) as end_date,
  CAST(NULL AS INT) as start_year,
  CAST(NULL AS INT) as end_year,
  
  -- Lead investigator (not available in this source)
  CAST(NULL AS STRUCT<
    given_name:STRING,
    family_name:STRING,
    orcid:STRING,
    role_start:DATE,
    affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
  >) as lead_investigator,
  
  -- Co-lead investigator (not available in this source)
  CAST(NULL AS STRUCT<
    given_name:STRING,
    family_name:STRING,
    orcid:STRING,
    role_start:DATE,
    affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
  >) as co_lead_investigator,
  
  -- Investigators (not available in this source)
  CAST(NULL AS ARRAY<STRUCT<
    given_name:STRING,
    family_name:STRING,
    orcid:STRING,
    role_start:DATE,
    affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
  >>) as investigators,
  
  -- Landing page URL (not available in this source)
  CAST(NULL AS STRING) as landing_page_url,
  
  -- DOI (not available in this source)
  CAST(NULL AS STRING) as doi,
  
  -- Works API URL
  concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(concat(f.funder_id, ':', a.normalized_award_id))) % 9000000000) as works_api_url,
  
  -- Timestamps
  current_timestamp() as created_date,
  current_timestamp() as updated_date
  
FROM awards a
JOIN funders f ON f.funder_id = a.funder_id

In [ ]:
%sql
-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    2 as priority
FROM openalex.awards.backfill_awards;